# Assigment 2

## The assignment is divided into programming and mathematical questions. Both of them are given in this notebook.

## Programming questions: I am giving you a template that you can use to write your code. Description of the questions is integrated in the comments.

## Upload your code on Learn dropbox and submit pdfs of the code and answers to the mathematical questions on Crowdmark.

## -----------------------------------------------------------------------------------------------------------

## Load modules

In [ ]:
# !pip install numpy, scipy, scikit-image, skimage, matplotlib

import matplotlib.pyplot as plt

from skimage.color import rgb2gray
from skimage import data
from skimage.transform import resize

# Numpy is useful for handling arrays and matrices.
import numpy as np
import time

## Load image

In [ ]:
img = data.astronaut()
img = rgb2gray(img)*255 # convert to gray and change scale from (0,1) to (0,255).

n = img.shape[0]
m = n

plt.figure(1, figsize=(10, 10))
plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.show()

## Compute the differences operators here. Use your code from Assignment 1.

In [ ]:
# You will need these three methods to construct sparse differences operators.
# If you do not use sparse operators you might have scalability problems.
from scipy.sparse import diags
from scipy.sparse import kron
from scipy.sparse import identity
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
from scipy import real

# Use your code from Assignment 1. 
# Make sure that you compute the right D_h and D_v matrices.

J = diags(diagonals=[
    [-1.0] * n,
    [1.0] * (n-1)
], offsets=[0, 1])

I = identity(n=n)

# Forward Horizontal Difference
D_h = kron(J, I)

# Forward Vertical Difference
D_v = kron(I, J)

D = D_h + 1j * D_v

D_s = real((D.H).dot(D))
I_s = identity(n*m)


# Matrix norm with tolerance parameter to control speed vs. precision tradeoff
def matrix_p2_norm(A, tol=0.001):
    eig_val = eigsh((A.H).dot(A), return_eigenvectors=False, tol=tol, k=1)
    return np.sqrt(eig_val).sum()

# Vector norm
def vector_norm(x):
    return np.sqrt(real((x.H).dot(x))).sum()

# The denoising objective function Hessian
def G(lamb):
    return lamb * D_s + I_s

# The denoising objective function
def f(lamb, x, z_n):
    return real(0.5 * lamb * (vector_norm(D.dot(x)) ** 2.0) + 0.5 * vector_norm(x - z_n) ** 2.0)

# The denoising objective function gradient
def grad_f(lamb, x, z_n):
    return G(lamb).dot(x) - z_n

## Add noise to the image

In [ ]:
mean_ = 0
standard_deviation = 30
dimensions = (n,n)

noise = np.random.normal(mean_,standard_deviation,dimensions)

noisy_image = img + noise
noisy_image_vec = csr_matrix(noisy_image.T.ravel()).T

plt.figure(1, figsize=(10, 10))
plt.imshow(noisy_image, cmap='gray', vmin=0, vmax=255)
plt.show()

 ## Question 1: implement gradient descent using the Lipschitz constant as the step-size for the denoising problem. Use eigsh method from scipy.sparse.linalg to compute the Lipschitz constant. Marks: 10

In [ ]:
def gradient_descent_lip(x0, epsilon, lambda_, max_iterations):
    # x0: is the initial guess for the x variables
    # epsilon: is the termination tolerance parameter
    # lambda_: is the regularization parameter of the denoising problem.
    # max_iterations: is the maximum number of iterations that you allow the algorithm to run.

    # Write your code here.
    l_t1 = time.time()
    L = matrix_p2_norm(G(lamb=lambda_), tol=0)
    l_t2 = time.time()
    print(f"Lipschitz Constant = {L}")
    x_updated = x0.copy()
    f_vals = []
    norm_vals = []
    it_t1 = time.time()
    for i in range(1, max_iterations+1):
        current_grad = grad_f(lamb=lambda_, x=x_updated, z_n=noisy_image_vec)
        current_grad_norm = vector_norm(current_grad)
        if current_grad_norm <= epsilon:
            break
        norm_vals.append(current_grad_norm)
        f_vals.append(f(lamb=lambda_, x=x_updated, z_n=noisy_image_vec))
        x_updated = x_updated - (1.0 / L) * current_grad
        f_diff = (f_vals[-1] - f_vals[-2]) if len(f_vals) > 1 else None
        grad_diff = (norm_vals[-1] - norm_vals[-2]) if len(norm_vals) > 1 else None
        print(f"Step = {i}: Function = {f_vals[-1]}, Function Diff. =  {f_diff}, Grad. Norm = {norm_vals[-1]}, Grad. Diff. = {grad_diff}")
    it_t2 = time.time()
    print(f"Iterations (Total) time = {it_t2-it_t1}, Lip-Constant time = {l_t2-l_t1}")
    return x_updated, np.array(f_vals)

## Call Gradient Descent

In [ ]:
# Initialize parameters of gradient descent.
lambda_ = 4
epsilon = 1.0e-2
max_iterations = 2000

# Set x0 equal to the vectorized noisy image.
# Write your code here.
optimized_gd_lip, f_vals_lip = gradient_descent_lip(x0=noisy_image_vec, 
                                                    lambda_=lambda_, 
                                                    epsilon=epsilon, 
                                                    max_iterations=max_iterations)

## Plot $$(f(x_k) - f(x^*)) / (f(x_0) - f(x^*))$$ vs the iteration counter k, where $$x^*$$ is the minimizer of the denoising problem, which you can compute by using spsolve, similarly to Assignment 1.

In [ ]:
# Finding the optimal solution
from scipy.sparse.linalg import spsolve
x_star = spsolve(A=G(lamb=lambda_), b=noisy_image_vec)
f_star = f(lamb=lambda_, x=csr_matrix(x_star).T, z_n=noisy_image_vec)
denoised_image_star = x_star.reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_star, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
# Plot the relative objective function vs number of iterations. 
rate_lip = (f_vals_lip - f_star) / (f_vals_lip[0] - f_star)

denoised_image_gd_lip = optimized_gd_lip.toarray().reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_gd_lip, cmap='gray', vmin=0, vmax=255)
plt.show()

print(f"Max. Abs. Diff. between GD-Lip and Linear Solver = {np.abs(diff_lip).max()}")

fig = plt.figure(figsize=(8, 6))
plt.plot(rate_lip, label=("Gradient descent + Lip."), linewidth=5.0, color ="black")
plt.legend(prop={'size': 20},loc="upper right")
plt.xlabel("iteration $k$", fontsize=25)
plt.ylabel("Relative distance to opt.", fontsize=25)
plt.grid(linestyle='dashed')
plt.show()


fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(rate_lip, label=("Gradient descent + Lip."), linewidth=5.0, color ="black")
ax.set_yscale('log')
plt.legend(prop={'size': 20},loc="upper right")
plt.xlabel("iteration $k$", fontsize=25)
plt.ylabel("Relative distance to opt. (LOG)", fontsize=25)
plt.grid(linestyle='dashed')
plt.show()

## Question 2: is there a "gap" between the practical convergence rate and the theoretical convergence rate? Note that the denoising objective function is strongly convex. Marks: 5

Yes. The practical convergence rate is significantly better than the theoritical convergence rate (i.e. the algorithm takes much less time than the theoritical bound), especially in the first few iterations (as seen on the log-scale curve)

Since the denoising objective function is strongly convex, the theoritical iteration complexity can be proven to be $k \geq \frac{L}{\mu} \log(\frac{f(x_0) - f^*}{\epsilon})$.

Substiuting the values of $L \approx 33, \mu \approx 1, \epsilon = 0.01$ (See question 7 to see how $\mu$ is computed, and using the implemetation above to compute $f(x_0)$ and $f^*$, we can see that $k > \thicksim850$ iterations! While in this experiment, the algorithm converges after $\thicksim 350$ iterations.

The reason is that, as mentioned in the lecture, these bounds on convergence rates are based on worst-case analysis, which might not necessiarily be the case in the denoising problem with this particular image. A more realistic estimate might be obtained by doing an average case analysis of the algorithm.

## Question 3: implement gradient descent with line-search for the denoising problem. Marks: 15

In [ ]:
# Write a line-search-ls function here.  
def line_search_ls(lambda_, x, f_x, grad_f_x):
    # lambda_: the regularization parameter
    # x: the current estimate of t=he variable
    # f_x: the value of the objective function at x
    # grad_f_x: the gradient of the objective function at x
    
    # Write your code here.
    alpha = 1.0
    while f(lamb=lambda_, x=x-alpha*grad_f_x, z_n=noisy_image_vec) >= f_x:
        alpha /= 2.0
    return alpha

# Write gradient descent + line-search here.
def gradient_descent_ls(x0, epsilon, lambda_, max_iterations):
    # x0: is the initial guess for the x variables
    # epsilon: is the termination tolerance parameter
    # lambda_: is the regularization parameter of the denoising problem.
    # max_iterations: is the maximum number of iterations that you allow the algorithm to run.

    # Write your code here.
    x_updated = x0.copy()
    f_vals = []
    norm_vals = []
    t1 = time.time()
    for i in range(1, max_iterations+1):
        current_grad = grad_f(lamb=lambda_, x=x_updated, z_n=noisy_image_vec)
        current_grad_norm = vector_norm(current_grad)
        if current_grad_norm <= epsilon:
            break
        norm_vals.append(current_grad_norm)
        f_vals.append(f(lamb=lambda_, x=x_updated, z_n=noisy_image_vec))
        alpha = line_search_ls(lambda_=lambda_, x=x_updated, f_x=f_vals[-1], grad_f_x=current_grad)
        x_updated = x_updated - alpha * current_grad
        f_diff = (f_vals[-1] - f_vals[-2]) if len(f_vals) > 1 else None
        grad_diff = (norm_vals[-1] - norm_vals[-2]) if len(norm_vals) > 1 else None
        print(f"Step = {i}: alpha = {alpha}, Function = {f_vals[-1]}, Function Diff. =  {f_diff}, Grad. Norm = {norm_vals[-1]}, Grad. Diff. = {grad_diff}")
    t2 = time.time()
    print(f"Iterations (Total) time = {t2-t1}")
    return x_updated, np.array(f_vals)

## Call Gradient Descent with line-search

In [ ]:
# Initialize parameters of gradient descent
lambda_ = 4
epsilon = 1.0e-2
max_iterations = 2000

# Set x0 equal to the vectorized noisy image.
# Write your code here.
optimized_gd_ls, f_vals_ls = gradient_descent_ls(x0=noisy_image_vec, 
                                                 lambda_=lambda_, 
                                                 epsilon=epsilon, 
                                                 max_iterations=max_iterations)

## Plot $$(f(x_k) - f(x^*)) / (f(x_0) - f(x^*))$$ vs the iteration counter k, where $$x^*$$ is the minimizer of the denoising problem, which you can compute by using spsolve, similarly to Assignment 1.

In [ ]:
# Write your code here.

# Plot the rellative objective function vs number of iterations. 
rate_ls = (f_vals_ls - f_star) / (f_vals_ls[0] - f_star)

denoised_image_gd_ls = optimized_gd_ls.toarray().reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_gd_ls, cmap='gray', vmin=0, vmax=255)
plt.show()

print(f"Max. Abs. Diff. between GD-LS and Linear Solver = {np.abs(diff_ls).max()}")

fig = plt.figure(figsize=(8, 6))
plt.plot(rate_ls, label=("Gradient descent + LS"), linewidth=5.0, color ="black")
plt.legend(prop={'size': 20},loc="upper right")
plt.xlabel("iteration $k$", fontsize=25)
plt.ylabel("Relative distance to opt.", fontsize=25)
plt.grid(linestyle='dashed')
plt.show()

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(rate_ls, label=("Gradient descent + LS"), linewidth=5.0, color ="black")
ax.set_yscale('log')
plt.legend(prop={'size': 20},loc="upper right")
plt.xlabel("iteration $k$", fontsize=25)
plt.ylabel("Relative distance to opt. (LOG)", fontsize=25)
plt.grid(linestyle='dashed')
plt.show()

## Question 4: What is the advantage of using line-search to compute the step-size at each iteration instead of using constant step-sizes equal to 1/L? Where L is the Lipschitz constant. Is gradient descent with line-search faster than gradient descent with constant step-sizes in terms of running time? Is gradient descent with line-search faster than gradient descent with constant step-sizes in terms of running time when you add computation of the Lipschitz constant in the running time? Is gradient descent with line-search faster than gradient descent with constant step-sizes in terms of number of required iterations? Marks: 10

Question breakdown (Please note that all the experimental numbers are reported on my machine and might differ from any other machine, but the main conclusion should still hold):

#### What is the advantage of using line-search to compute the step-size at each iteration instead of using constant step-sizes equal to 1/L? Where L is the Lipschitz constant. 

Computing the Lipschitz constant is a very expensive operation especially on large number of parameters. So, the advantage is to replace that expensive operation with a less costly one that still guarantees the decrease of the objective function at each iteration.

#### Is gradient descent with line-search faster than gradient descent with constant step-sizes in terms of running time?

Yes. Constant-step GD takes 198 seconds in total, where GD with line search takes 15.6 seconds in total. Both reaching the same tolernace as specified by the parameter epsilon.

#### Is gradient descent with line-search faster than gradient descent with constant step-sizes in terms of running time when you add computation of the Lipschitz constant in the running time?

Yes. Constant-step GD takes 26 seconds in total iterations time, and takes 172 seconds in computing the Lipschitz constant, where GD with line search takes 15.6 seconds in total. Both reaching the same tolernace as specified by the parameter epsilon.

#### Is gradient descent with line-search faster than gradient descent with constant step-sizes in terms of number of required iterations?

Yes. Constant-step GD takes 357 iterations, where GD with line search takes 91 iterations. Both reaching the same tolernace as specified by the parameter epsilon.

## Questions 5: implement gradient descent with Armijo line-search for the denoising problem. Marks: 10

In [ ]:
# Create a line-search-armijo function
def line_search_arm(lambda_, x, f_x, grad_f_x, norm_grad_f_x, gamma):
    # lambda_: the regularization parameter
    # x: the current estimate of t=he variable
    # f_x: the value of the objective function at x
    # grad_f_x: the gradient of the objective function at x
    # norm_grad_f_x: the norm of grad_f_x
    # gamma: parameter of Armijo line-search as was defined in the lectures.

    # Write your code here.
    alpha = 1.0
    loss = gamma * (norm_grad_f_x ** 2.0)
    while f(lamb=lambda_, x=x-alpha*grad_f_x, z_n=noisy_image_vec) > f_x - alpha * loss:
        alpha /= 2.0
    return alpha
    
def gradient_descent_arm(x0, epsilon, lambda_, max_iterations, gamma):
    # x0: is the initial guess for the x variables
    # epsilon: is the termination tolerance parameter
    # lambda_: is the regularization parameter of the denoising problem.
    # max_iterations: is the maximum number of iterations that you allow the algorithm to run.
    # gamma: parameter of Armijo line-search as was defined in the lectures.
    
    # Write your code here.
    x_updated = x0.copy()
    f_vals = []
    norm_vals = []
    t1 = time.time()
    for i in range(1, max_iterations+1):
        current_grad = grad_f(lamb=lambda_, x=x_updated, z_n=noisy_image_vec)
        current_grad_norm = vector_norm(current_grad)
        if current_grad_norm <= epsilon:
            break
        norm_vals.append(current_grad_norm)
        f_vals.append(f(lamb=lambda_, x=x_updated, z_n=noisy_image_vec))
        alpha = line_search_arm(lambda_=lambda_, x=x_updated, f_x=f_vals[-1], grad_f_x=current_grad, norm_grad_f_x=current_grad_norm, gamma=gamma)
        x_updated = x_updated - alpha * current_grad
        f_diff = (f_vals[-1] - f_vals[-2]) if len(f_vals) > 1 else None
        grad_diff = (norm_vals[-1] - norm_vals[-2]) if len(norm_vals) > 1 else None
        print(f"Step = {i}: alpha = {alpha}, Function = {f_vals[-1]}, Function Diff. =  {f_diff}, Grad. Norm = {norm_vals[-1]}, Grad. Diff. = {grad_diff}")
    t2 = time.time()
    print(f"Iterations (Total) time = {t2-t1}")
    return x_updated, np.array(f_vals)

## Call Gradient Descent with Armijo line search

In [ ]:
# Initialize parameters of gradient descent
lambda_ = 4
epsilon = 1.0e-2
gamma = 0.13
max_iterations = 2000

# Set x0 equal to the vectorized noisy image.
# Write your code here.
optimized_gd_arm, f_vals_arm = gradient_descent_arm(x0=noisy_image_vec, 
                                                    lambda_=lambda_, 
                                                    epsilon=epsilon,
                                                    gamma=gamma,
                                                    max_iterations=max_iterations)

## Plot $$(f(x_k) - f(x^*)) / (f(x_0) - f(x^*))$$ vs the iteration counter k, where $$x^*$$ is the minimizer of the denoising problem, which you can compute by using spsolve, similarly to Assignment 1.

In [ ]:
# Plot the relative objective function vs number of iterations. 

rate_arm = (f_vals_arm - f_star) / (f_vals_arm[0] - f_star)

denoised_image_gd_arm = optimized_gd_arm.toarray().reshape((m, n), order='F')
plt.figure(1, figsize=(10, 10))
plt.imshow(denoised_image_gd_arm, cmap='gray', vmin=0, vmax=255)
plt.show()

print(f"Max. Abs. Diff. between GD-ARM and Linear Solver = {np.abs(diff_arm).max()}")

fig = plt.figure(figsize=(8, 6))
plt.plot(rate_arm, label=("Gradient descent + ARM LS"), linewidth=5.0, color ="black")
plt.legend(prop={'size': 20},loc="upper right")
plt.xlabel("iteration $k$", fontsize=25)
plt.ylabel("Relative distance to opt.", fontsize=25)
plt.grid(linestyle='dashed')
plt.show()

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(rate_arm, label=("Gradient descent + ARM LS"), linewidth=5.0, color ="black")
ax.set_yscale('log')
plt.legend(prop={'size': 20},loc="upper right")
plt.xlabel("iteration $k$", fontsize=25)
plt.ylabel("Relative distance to opt. (LOG)", fontsize=25)
plt.grid(linestyle='dashed')
plt.show()

## Question 6: Is gradient descent with Armijo line-search faster than gradient descent with simple line-search in terms of running time? Is gradient descent with Armijo line-search faster than gradient descent with simple line-search in terms of number of required iterations? Explain any performance differences between the two approaches. Marks: 10

Question breakdown (Please note that all the experimental numbers are reported on my machine and might differ from any other machine, but the main conclusion should still hold):

#### Is gradient descent with Armijo line-search faster than gradient descent with simple line-search in terms of running time?
Yes. GD with simple line search takes 15.6 seconds in total, where GD with Armijo line search with $\gamma = 0.13$ takes 14 seconds in total. Both reaching the same tolernace as specified by the parameter epsilon.

#### Is gradient descent with Armijo line-search faster than gradient descent with simple line-search in terms of number of required iterations? 

Yes. GD with simple line search takes 91 iterations, where GD with Armijo line search with $\gamma = 0.13$ takes 77 iterations in total. Both reaching the same tolernace as specified by the parameter epsilon. 

#### Explain any performance differences between the two approaches.

The main difference between the two methods is that the Armijo line search is more "agressive" in finding the $\alpha$ parameter than the simple line search. The check is done such that the objective function decreased sufficiently (by at least $\alpha \gamma ||\nabla f(x)||_2^2$), not just decreased as in simple line search. This may lead to more line search iterations (and hence a longer time per gradient descent step), but it will also lead to less total number of gradient descent steps, since the convex function is forced to be decreased more every iteration.

## Mathematical Questions

## Question 7: prove that the denoising objective function is strongly convex. What is its strong convexity parameter? Marks: 5

From assignment 1 we know that, for the denoising objective function:
$$ 
\nabla^2 f(x) = \lambda_{reg} (D^*.D) + I
$$

Where $\lambda_{reg}$ is the regularization parameter.

We know that the defintion of a strongly convex (twice-differentiable) function is:
$$
y^T.\nabla^2 f(x).y \geq \mu ||y||^2_2
$$

For now, let's call $G = \nabla^2 f(x) = \lambda_{reg} (D^*.D) + I$. Which is a constant that does not depend on $x$.

We then rewrite the strong convexity condition as: $y^T.G.y \geq \mu ||y||^2_2$

Starting with left-hand side, we replace the product $G.y$ with $\lambda y$, where $\lambda$ is an eigenvalue of the matrix $G$. And noticing that $y^T.y = ||y||_2^2$. We then have:

$$
y^T.G.y =
y^T.\lambda.y =
\lambda.(y^T.y) =
\lambda ||y||_2^2 \geq \mu ||y||^2_2
$$


So, the denoising objective function is strongly convex, and the strong convexity parameter is given by the smallest positive eigenvalue of the hessian (Which will always exist given that $G$ is positive semi-definite, so all eigenvalues are greater than or equal to zero).

Using the function $eigsh$ with $\lambda = 4.0$ to compute the strong convexity parameter, we find that $\mu = \lambda_{min} \approx 1$

## Question 8: Prove that Armijo line-search will terminate after a finite number of steps. Hint: show that there exists a step-size $$\alpha^*>0$$ such that for any step-size smaller than $$\alpha^*$$ the termination condition of Armijo line-search is satisfied. How many iterations will be required in worst-case for Armijo line-search to terminate? Marks 15

Uploaded to Learn separately.

## Question 9: what is the running time for gradient descent with Armijo line-search for the denoising problem to achieve $$f(x_k) - f^* \le \epsilon$$ ?. The running time is computed by multiplying the worst-case iteration complexity times the FLOPS at each iteration. The FLOPS at each iteration is the number of additions, subtractions, multiplications and divisions that are performed during the current iteration. 10

Uploaded to Learn separately.

## Question 10: prove the convergence rate and iteration complexity for gradient descent with constant step-sizes (equal to 1/L) for strongly convex functions. Marks: 10

Uploaded to Learn separately.